# Homework 05 - More on DNNs

In this homework you will deepen your understanding of TensorFlow. You will implement a simple classifier for the MNIST dataset.    
 
Be aware that sometimes there are variable names in the provided code that need to align with variables that you define. You can either change my or your code.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib notebook

### Load the dataset.

In [ ]:
# TensorFlow has some popular datasets already included. 
# Check out https://www.tensorflow.org/api_docs/python/tf/keras/datasets.
# In this homework we will work with the popular MNIST dataset.
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

### Investigate the dataset.

**It is always important to understand the format and the count of the data you are dealing with.**  
**Please answer the following questions:**

Q: *How many training/test images does this dataset have?*  
A: 60000 training images, 10000 test images

Q: *Which shape do the images have?*  
A: 28 times 28 pixels

Q: *In which range are the pixel values?*  
A: grey-scale from 0 to 255

In [ ]:
# You should make use of the following functions and properties.
# You can access the shape of an array 'arr' with 'arr.shape'.
# You can use 'np.max', 'np.min' to access the maximum or minimum of an array.
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)
print(np.max(train_images[0]))
print(np.min(train_images[0]))

### Visualize the dataset.

In [ ]:
# It can also be helpful to have a proper look at specific samples of a dataset to understand what you are dealing with.
# Plot 5 samples with their corresponding label.

fig, ax = plt.subplots(1,5)
for i in range(5):
    ### YOUR CODE HERE ###
    # Readout an image and the corresponding label.
    img = train_images[i]
    lbl = train_labels[i]
    ######################
    ax[i].imshow(img, cmap='gray')
    ax[i].set_title(lbl)
    ax[i].axis("off")

### Build the TensorFlow dataset.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# It makes sense to shuffle the training dataset.
train_dataset = train_dataset.shuffle(buffer_size=60000)
# Further we have seen that it is advantageous to train with mini-batches instead of one sample at a time.
train_dataset = train_dataset.batch(128)
# Let's have a look at how the elements of the dataset now look.
for elem in train_dataset:
    print(elem[0].shape)
    print(elem[1].shape)
    break
# The test dataset can be processed in full batch.
test_dataset = test_dataset.batch(10000)

### Build the model.

In [ ]:
# We will implement a simple fully connected feed forward neural network like the last time.
#
# Our network will have the following layers:
# Input : 784 units.
# Hidden layer 1: 256 units. With sigmoid activation function.
# Hidden layer 2: 256 units. With sigmoid activation function.
# Output: 10 units. With softmax activation function.
# 
# Instead of implementing our own layer we can directly implement the network using pre-built layers 
# from TensorFlow.
# For that check out 'tf.keras.layers.Dense(units= , activation=)'. It is basically the same layer that we
# implemented by hand last time.
# For activations functions check out 'tf.keras.activations'.

from tensorflow.keras.layers import Layer

class Model(Layer):
    
    def __init__(self):
        super(Model, self).__init__()
        ### YOUR CODE HERE ###
        # Define the three layers.
        self.hidden_layer_1 = tf.keras.layers.Dense(units=256,
                                               activation=tf.keras.activations.sigmoid
                                               )
        self.hidden_layer_2 = tf.keras.layers.Dense(units=256,
                                               activation=tf.keras.activations.sigmoid
                                               )
        self.output_layer = tf.keras.layers.Dense(units=10,
                                               activation=tf.keras.activations.softmax
                                               )
        ######################
        
    def call(self, x):
        ### YOUR CODE HERE ###
        # Define the forward step.
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.output_layer(x)
        ######################
        return x

### Question:

*Q: How many parameters does this network have?*  
A: 784x256 + 256 + 256x256 + 256 + 256x10 + 10 = 269322

### Train the model.

In [ ]:
tf.keras.backend.clear_session()

### YOUR CODE HERE ###
# Initialize the model.
model = Model()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam()
######################

# Initialize lists for later visualization.
train_steps = []
train_losses = []
train_accuracies = []
test_steps = []
test_losses = []
test_accuracies = []
step = 0


# We train for 3 epochs.
for epoch in range(3):
    
    for (x,t) in train_dataset:
        
        ### YOUR CODE HERE ###
        # We need to reshape the input from [batchsize,28,28] to [batchsize,784].
        x = tf.reshape(x, shape=(-1,784))
        # Further we need to transform the labels into a one-hot encoding.
        # Check out 'tf.one_hot'.
        t = tf.one_hot(t, depth=10)
        ######################
        
        ### YOUR CODE HERE ###
        # Compute the output, loss and the gradients.
        with tf.GradientTape() as tape:
            output = model(x)
            loss = cross_entropy_loss(t, output)
            gradients = tape.gradient(loss, model.trainable_variables)
        # Apply gradients.
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  
        ######################
        
        # Compute the accuracy.
        accuracy = np.sum(np.argmax(t, axis=1) == np.argmax(output, axis=1)) / t.shape[0]
        # Store loss and accuracy.
        train_accuracies.append(accuracy)
        train_losses.append(loss)
        
        # After each 25 training step we check the generalization to the test dataset.
        if step % 25 == 0:
            for (x,t) in test_dataset:
                ### YOUR CODE HERE ###
                x = tf.reshape(x, shape=(-1,784))
                t = tf.one_hot(t, depth=10)
                output = model(x)
                loss = cross_entropy_loss(t, output)
                accuracy = np.sum(np.argmax(t, axis=1) == np.argmax(output, axis=1)) / t.shape[0]
                ######################
                test_steps.append(step)
                test_accuracies.append(accuracy)
                test_losses.append(loss)
        
        train_steps.append(step)
        step += 1

### Visualize the training progress.

In [ ]:
# Visualize accuracy and loss for training and test data. 
# One plot training and test loss.
# One plot training and test accuracy.
### YOUR CODE HERE###
plt.figure()
line1, = plt.plot(train_steps, train_losses)
line2, = plt.plot(test_steps, test_losses)
plt.xlabel("Training steps")
plt.ylabel("Loss")
plt.legend((line1,line2),("training","test"))
plt.show()

plt.figure()
line1, = plt.plot(train_steps, train_accuracies)
line2, = plt.plot(test_steps, test_accuracies)
plt.xlabel("Training steps")
plt.ylabel("Accuracy")
plt.legend((line1,line2),("training","test"))
plt.show()
#####################